In [50]:
import cv2
import numpy as np
import face_recognition as fr
import os
import pandas as pd
from datetime import datetime
import pyttsx3
import threading
import csv

In [ ]:
path='images'

In [53]:
os.listdir(path)

['Dustin.jpg', 'Eleven.jpg', 'Mike.jpg', 'Steve.jpg']

In [54]:
mylist=os.listdir(path)

In [55]:
imgs=[]
classnames=[]
for i in mylist:
    imgpath=os.path.join(path,i)
    img1=cv2.imread(imgpath)
    imgs.append(img1)

    classnames.append(i.split('.')[0])

In [56]:
print(classnames)

['Dustin', 'Eleven', 'Mike', 'Steve']


In [57]:
def facencodings(images):
    encodelist=[]
    for i in imgs:
        img1=cv2.cvtColor(i,cv2.COLOR_BGR2RGB)
        face_in_frame=fr.face_locations(img1)
        #print(face_in_frame)
        face_encode=fr.face_encodings(img1,face_in_frame)[0]
        #print(face_encode)
        encodelist.append(face_encode)
    return encodelist


In [58]:
encodelist_knownfaces=facencodings(imgs)
print(encodelist_knownfaces)

[array([-0.13153863,  0.01305345,  0.12305073, -0.09447822, -0.13175064,
       -0.08696323, -0.07570942,  0.01852353,  0.11594161, -0.05723464,
        0.14839236, -0.07799566, -0.19750015, -0.05562029, -0.0112239 ,
        0.15378542, -0.15190999, -0.10896804, -0.09956798, -0.03668071,
        0.04082414, -0.00574351, -0.0057471 ,  0.10081787, -0.05031537,
       -0.355266  , -0.06665945, -0.15340151,  0.03409415, -0.08504406,
       -0.01173165,  0.02390589, -0.1808202 , -0.09054641,  0.02019284,
       -0.00771993, -0.07180403, -0.05831438,  0.23833092,  0.04463507,
       -0.14089806, -0.00800906, -0.00661169,  0.23727863,  0.19488224,
        0.10300044,  0.0224717 , -0.03386223,  0.15678248, -0.24127027,
        0.03708417,  0.20321602,  0.10173421,  0.03663119,  0.18663515,
       -0.08958482,  0.12290021,  0.16268897, -0.27518651,  0.1151263 ,
       -0.00747293, -0.04711187,  0.05166768, -0.07729073,  0.21768196,
        0.0212354 , -0.06157412, -0.06985839,  0.16635831, -0.1

In [ ]:
csv_file = 'attendance.csv'
with open(csv_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Name', 'Date','Time'])
attendence_marked=set()

In [60]:
# Function to speak in a separate thread
def speak_name(name):
    txt_sp = pyttsx3.init()
    voices = txt_sp.getProperty('voices')
    txt_sp.setProperty('voice', voices[0].id)
    text = f"Attendance marked for {name}"
    txt_sp.say(text)
    txt_sp.runAndWait()

In [ ]:
video=cv2.VideoCapture(0)
while True:
    suc,frame=video.read()
    frame1=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    face_in_frame=fr.face_locations(frame1)
    face_encode=fr.face_encodings(frame1,face_in_frame)
    for enc_face,loc_face in zip(face_encode,face_in_frame):
        matches=fr.compare_faces(encodelist_knownfaces,enc_face)
        face_dis=fr.face_distance(encodelist_knownfaces,enc_face)
        
        if len(face_dis)>0:
            match_index=np.argmin(face_dis)
            
            if matches[match_index]:
                name=classnames[match_index]
                
                if name not in attendence_marked:

                    attendence_marked.add(name)

                    today = datetime.now().strftime("%Y-%m-%d")

                    now = datetime.now().strftime('%H:%M:%S')
                    with open(csv_file, 'a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([name,now,today])

                    print(f'Attendance marked for {name} at {now}')

                    threading.Thread(target=speak_name, args=(name,), daemon=True).start()
                    
            else:
                name="Unknown Face"

        else:
            name="Unknown Face"

        # Draw rectangle and name on frame
        y1,x2,y2,x1=loc_face
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
        cv2.putText(frame,name,(x1,y1-10),cv2.FONT_HERSHEY_COMPLEX,0.8,(255,0,0),2)
    
    cv2.imshow('img',frame)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

video.release()
cv2.destroyAllWindows()


Attendance marked for Dustin at 11:39:18
Attendance marked for Mike at 11:39:24
Attendance marked for Eleven at 11:39:34
Attendance marked for Steve at 11:39:43
